# Gradient Boosting classifier with decision trees

Gradient boosting is an ensemble learning technique that consists of sequentially adding predictors. Each predictor will fit on the residual error of the previous predictor.

The predictors used in this case are weak learners decision trees.

In [9]:
library(xgboost)
library(caret)

In [10]:
column_to_drop<-c("X.1","X")
training_set <- read.csv("../Data/PreProcess/processed_training_data_split.csv") # load training set
validation_set <- read.csv("../Data/PreProcess/processed_verification_data_split.csv") # load validation set

# Dropped "X.1","X" because they just represent the row numbers
training_set<-training_set[,!(names(validation_set) %in% column_to_drop)] # drop the desired columns on validation set
validation_set<-validation_set[,!(names(validation_set) %in% column_to_drop)] # drop the desired columns on validation set

In [ ]:
## test GBM - fait planter le kernel 
library(gbm)
gbm.fit <- gbm(
  formula = id ~ .,
  distribution = "gaussian",
  data = training_set,
  n.trees = 10,
  interaction.depth = 1,
  shrinkage = 0.9,
  cv.folds = 5,
  verbose = FALSE
  )  

Using  trees...




In [11]:
## test xgboost
modelGB <- train(
  id ~., data = training_set, method = "xgbTree",
  trControl = trainControl("cv", number = 3)
  )
# tuning parameter
modelGB$bestTune

,nrounds,max_depth,eta,gamma,colsample_bytree,min_child_weight,subsample
,<dbl>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
102,150,3,0.4,0,0.8,1,0.5


In [12]:
modelGB
save(modelGB,file = "4-Models/modelGB.RData")

eXtreme Gradient Boosting 

47289 samples
   83 predictor
    3 classes: 'functional', 'functional needs repair', 'non functional' 

No pre-processing
Resampling: Cross-Validated (3 fold) 
Summary of sample sizes: 31527, 31526, 31525 
Resampling results across tuning parameters:

  eta  max_depth  colsample_bytree  subsample  nrounds  Accuracy   Kappa    
  0.3  1          0.6               0.50        50      0.7125337  0.4177527
  0.3  1          0.6               0.50       100      0.7206963  0.4398183
  0.3  1          0.6               0.50       150      0.7240163  0.4499367
  0.3  1          0.6               0.75        50      0.7122800  0.4166842
  0.3  1          0.6               0.75       100      0.7199561  0.4376988
  0.3  1          0.6               0.75       150      0.7241855  0.4490711
  0.3  1          0.6               1.00        50      0.7104614  0.4130898
  0.3  1          0.6               1.00       100      0.7194064  0.4357767
  0.3  1          0.6     

In [13]:
load("4-Models/modelGB.RData")
pred <- predict(modelGB,validation_set)
conf <- confusionMatrix(data=pred, reference = factor(validation_set$id[-1])) # GBM confusion matrix
conf

Confusion Matrix and Statistics

                         Reference
Prediction                functional functional needs repair non functional
  functional                    5446                     630           1960
  functional needs repair         58                      83             36
  non functional                1019                     204           2674

Overall Statistics
                                         
               Accuracy : 0.6774         
                 95% CI : (0.669, 0.6857)
    No Information Rate : 0.5386         
    P-Value [Acc > NIR] : < 2.2e-16      
                                         
                  Kappa : 0.3764         
                                         
 Mcnemar's Test P-Value : < 2.2e-16      

Statistics by Class:

                     Class: functional Class: functional needs repair
Sensitivity                     0.8349                       0.090513
Specificity                     0.5364                       0.9916